In [1]:
import mpp_classifiers as mpp
import load_ship_data as lsd
import numpy as np
import time
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold

MPP notebook

get data via our data loader

In [2]:

data_path="data/shipsnet.json"
train,test,valid=lsd.load_data_train_test_split(data_path)
def conf(pred,y):
    T0=sum([1 if x==y and y==0 else 0 for (x,y) in zip(pred,y)])
    T1=sum([1 if x==y and y==1 else 0 for (x,y) in zip(pred,y)])
    F0=sum([1 if x!=y and y==0 else 0 for (x,y) in zip(pred,y)])
    F1=sum([1 if x!=y and y==1 else 0 for (x,y) in zip(pred,y)])
    return np.array([[T0,F0],[F1,T1]])

Training Set Data Length:  2800   Label Length:  2800
TestingSet Set Data Length:  600  Label Length:  600
Validation Set Data Length:  600  Label Length:  600


Setup data splits and priors

Note: we split into train/valid/test as above, then we merge train+valid

This arrangement is due to scikit learn handling validation for us, a feature we were unaware of when designing data loader

In [3]:
ship_prob=np.sum(train[1])*1.0/train[1].shape[0]
p=np.array([1-ship_prob,ship_prob])
print(p)
Xtrain=train[0]
ytrain=train[1]
Xtest=test[0]
ytest=test[1]
Xvalid=valid[0]
yvalid=valid[1]
Xtrain=Xtrain.reshape(Xtrain.shape[0],Xtrain.shape[1]*Xtrain.shape[2]*Xtrain.shape[3])
Xtest=Xtest.reshape(Xtest.shape[0],Xtest.shape[1]*Xtest.shape[2]*Xtest.shape[3])
Xvalid=Xvalid.reshape(Xvalid.shape[0],Xvalid.shape[1]*Xvalid.shape[2]*Xvalid.shape[3])
print(Xtrain.shape)
print(Xtest.shape)
print(Xvalid.shape)
Xtrain=np.concatenate((Xtrain,Xvalid))
ytrain=np.concatenate((ytrain,yvalid))
print(Xtrain.shape)
print(ytrain.shape)

[0.7525 0.2475]
(2800, 19200)
(600, 19200)
(600, 19200)
(3400, 19200)
(3400,)


Case 1. No PCA, even priors

In [4]:

p55=np.array([.5,.5])
t0 = time.time()           # start time
y_model = mpp.mpp(Xtrain, ytrain, Xtest, 1, p55)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.67;
Classwise accuracy = [0.68584071 0.62162162];
The learning process takes 16.534067153930664 seconds.
[[310 142]
 [ 56  92]]


Case 2. No PCA, prior assumes ships are rare

In [5]:
p91=np.array([.9,.1])
t0 = time.time()           # start time
y_model = mpp.mpp(Xtrain, ytrain, Xtest, 1, p91)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.7533333333333333;
Classwise accuracy = [1. 0.];
The learning process takes 17.973310708999634 seconds.
[[452   0]
 [148   0]]


Case 1. No PCA, priors from training data

In [6]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(Xtrain, ytrain, Xtest, 1, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.7533333333333333;
Classwise accuracy = [1. 0.];
The learning process takes 18.152722120285034 seconds.
[[452   0]
 [148   0]]


PCA = 850 dimensions kept

In [7]:
pca = PCA(n_components=850)
pca.fit(Xtrain)
pca_Xtrain=pca.transform(Xtrain)
pca_Xtest=pca.transform(Xtest)
explained=np.sum(pca.explained_variance_ratio_)
print(explained)

0.9902384322929967


Case 1. PCA=850, training priors used from now on

In [8]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca_Xtrain, ytrain, pca_Xtest, 1, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.7533333333333333;
Classwise accuracy = [1. 0.];
The learning process takes 0.09625029563903809 seconds.
[[452   0]
 [148   0]]


Case 2. PCA=850

In [9]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca_Xtrain, ytrain, pca_Xtest, 2, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.9166666666666666;
Classwise accuracy = [0.94690265 0.82432432];
The learning process takes 38.98410153388977 seconds.
[[428  24]
 [ 26 122]]


Case 3. PCA=850

In [10]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca_Xtrain, ytrain, pca_Xtest, 3, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.7533333333333333;
Classwise accuracy = [1. 0.];
The learning process takes 54.95141100883484 seconds.
[[452   0]
 [148   0]]


PCA =245 dimensions kept

In [11]:
pca2 = PCA(n_components=245)
pca2.fit(Xtrain)
pca2_Xtrain=pca2.transform(Xtrain)
pca2_Xtest=pca2.transform(Xtest)
explained=np.sum(pca2.explained_variance_ratio_)
print(explained)

0.9501829567993927


Case 1. PCA=245

In [12]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca2_Xtrain, ytrain, pca2_Xtest, 1, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.7533333333333333;
Classwise accuracy = [1. 0.];
The learning process takes 0.04390287399291992 seconds.
[[452   0]
 [148   0]]


Case 2. PCA=245

In [13]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca2_Xtrain, ytrain, pca2_Xtest, 2, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.925;
Classwise accuracy = [0.96238938 0.81081081];
The learning process takes 1.7007274627685547 seconds.
[[435  17]
 [ 28 120]]


Case 3. PCA=245

In [14]:
p=np.array([1-ship_prob,ship_prob])
t0 = time.time()           # start time
y_model = mpp.mpp(pca2_Xtrain, ytrain, pca2_Xtest, 3, p)
t1 = time.time()           # ending time
acc_classwise, acc_overall = mpp.accuracy_score(ytest, y_model)
print(f'Overall accuracy = {acc_overall};')
print(f'Classwise accuracy = {acc_classwise};')
print(f'The learning process takes {t1 - t0} seconds.')
ar=conf(y_model,ytest)
print(ar)

Overall accuracy = 0.785;
Classwise accuracy = [0.78982301 0.77027027];
The learning process takes 3.0437214374542236 seconds.
[[357  95]
 [ 34 114]]


Setup for Kfold cross-validation

In [15]:
p=np.array([1-ship_prob,ship_prob])
X=pca2_Xtrain
y=ytrain
k=10

Case 1

In [16]:

kf=KFold(n_splits=k,random_state=None)
acc_scores=[]
for train_index,test_index in kf.split(X):
    X_train,X_test=X[train_index,:],X[test_index,:]
    y_train,y_test=y[train_index],y[test_index]
    y_model = mpp.mpp(X_train, y_train, X_test, 1, p)
    acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
    acc_scores.append(acc_overall)
avg_acc=sum(acc_scores)*1.0/k
print('Accuracy from each fold =  {}'.format(acc_scores))
print('Average Accuracy = {}'.format(avg_acc))

Accuracy from each fold =  [0.7588235294117647, 0.7588235294117647, 0.788235294117647, 0.7676470588235295, 0.7323529411764705, 0.7823529411764706, 0.7235294117647059, 0.7176470588235294, 0.7852941176470588, 0.7029411764705882]
Average Accuracy = 0.7517647058823529


Case 2

In [17]:
kf=KFold(n_splits=k,random_state=None)
acc_scores=[]
for train_index,test_index in kf.split(X):
    X_train,X_test=X[train_index,:],X[test_index,:]
    y_train,y_test=y[train_index],y[test_index]
    y_model = mpp.mpp(X_train, y_train, X_test, 2, p)
    acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
    acc_scores.append(acc_overall)
avg_acc=sum(acc_scores)*1.0/k
print('Accuracy from each fold =  {}'.format(acc_scores))
print('Average Accuracy = {}'.format(avg_acc))


Accuracy from each fold =  [0.9058823529411765, 0.9117647058823529, 0.9352941176470588, 0.9205882352941176, 0.9176470588235294, 0.9323529411764706, 0.9117647058823529, 0.9235294117647059, 0.9235294117647059, 0.9029411764705882]
Average Accuracy = 0.9185294117647059


Case 3

In [18]:
kf=KFold(n_splits=k,random_state=None)
acc_scores=[]
for train_index,test_index in kf.split(X):
    X_train,X_test=X[train_index,:],X[test_index,:]
    y_train,y_test=y[train_index],y[test_index]
    y_model = mpp.mpp(X_train, y_train, X_test, 3, p)
    acc_classwise, acc_overall = mpp.accuracy_score(y_test, y_model)
    acc_scores.append(acc_overall)
avg_acc=sum(acc_scores)*1.0/k
print('Accuracy from each fold =  {}'.format(acc_scores))
print('Average Accuracy = {}'.format(avg_acc))


Accuracy from each fold =  [0.8205882352941176, 0.7852941176470588, 0.8088235294117647, 0.7970588235294118, 0.7676470588235295, 0.7852941176470588, 0.8058823529411765, 0.7852941176470588, 0.8, 0.7911764705882353]
Average Accuracy = 0.7947058823529412
